Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Favg-bonus-201374%2Favg_bonus.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/avg-bonus-201374)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“What is the average total ability score bonus for the best of 4 DCC Characters?”](https://rpg.stackexchange.com/a/201379/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["ipycanvas==0.13.2", "ipyevents==2.0.1", "ipympl==0.9.4", "ipywidgets==8.1.3", "anydyce==0.4.6"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import H
from dyce.evaluation import HResult, foreach

d6 = H(6)
stat = 3@d6
stat_to_bonus_map = {
    3: -3,
    4: -2, 5: -2,
    6: -1, 7: -1, 8: -1,
    9: 0, 10: 0, 11: 0, 12: 0,
    13: +1, 14: +1, 15: +1,
    16: +2, 17: +2,
    18: +3,
}

def stat_to_bonus(stat: HResult) -> int:
    return stat_to_bonus_map[stat.outcome]

bonus_for_single_stat = foreach(stat_to_bonus, stat=stat)
print(f"bonus: {bonus_for_single_stat}")
print(f"bonus.total: {bonus_for_single_stat.total}")

bonus: H({-3: 1, -2: 9, -1: 46, 0: 104, 1: 46, 2: 9, 3: 1})
bonus.total: 216


In [3]:
from dyce import P
from anydyce import jupyter_visualize

bonus_for_six_stats = 6@bonus_for_single_stat
character_pool = 4@P(bonus_for_six_stats)
best_of_four_bonus_for_six_stats = character_pool.h(-1)

jupyter_visualize(
    [
        (f"Single Stat\nMean: {bonus_for_single_stat.mean():.2f}; Std Dev: {bonus_for_single_stat.stdev():.2f}", stat, bonus_for_single_stat),
        (f"All Six Stats Bonus\nMean: {bonus_for_six_stats.mean():.2f}; Std Dev: {bonus_for_six_stats.stdev():.2f}", bonus_for_six_stats),
        (f"Best-of-Four Bonus\nMean: {best_of_four_bonus_for_six_stats.mean():.2f}; Std Dev: {best_of_four_bonus_for_six_stats.stdev():.2f}", best_of_four_bonus_for_six_stats),
    ]
)